In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB

In [2]:
# Load your dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_ids = pd.read_csv("test.csv")["ID"]
# Analyze your dataset
# ID, Customer_ID, Month, Name, Age, Profession, Number are columns to be dropped
train_data = train_data.drop(columns=["ID", "Customer_ID", "Month", "Name", "Profession", "Number"])

print(train_data.head())

# Loan_Type, Credit_Mix, Payment_of_Min_Amount, Payment_Behaviour , and Credit_Score are categorical columns
# Credit_Score is the target column

C:\Users\krish\AppData\Local\Temp\ipykernel_10416\3242217444.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv("train.csv")


  Age Income_Annual  Base_Salary_PerMonth  Total_Bank_Accounts  \
0  51     101583.48                   NaN                    5   
1  23     101926.95             8635.9125                    4   
2  49     158871.12                   NaN                    0   
3  40      60379.28                   NaN                    5   
4  17      50050.83             4085.9025                    9   

   Total_Credit_Cards  Rate_Of_Interest Total_Current_Loans  \
0                   7                10                  4_   
1                   4                 9                   1   
2                   4                 8                   1   
3                   6                18                   3   
4                  10                20                   5   

                                           Loan_Type  Delay_from_due_date  \
0  Payday Loan, Student Loan, Payday Loan, and De...                    8   
1                                      Mortgage Loan                  

### Changes can be done here.Filling data with something other than median

In [3]:
# Fill missing values with median in train data
train_data.fillna(train_data.median(numeric_only=True), inplace=True)
print(train_data.head())

  Age Income_Annual  Base_Salary_PerMonth  Total_Bank_Accounts  \
0  51     101583.48           3086.683333                    5   
1  23     101926.95           8635.912500                    4   
2  49     158871.12           3086.683333                    0   
3  40      60379.28           3086.683333                    5   
4  17      50050.83           4085.902500                    9   

   Total_Credit_Cards  Rate_Of_Interest Total_Current_Loans  \
0                   7                10                  4_   
1                   4                 9                   1   
2                   4                 8                   1   
3                   6                18                   3   
4                  10                20                   5   

                                           Loan_Type  Delay_from_due_date  \
0  Payday Loan, Student Loan, Payday Loan, and De...                    8   
1                                      Mortgage Loan                  

In [4]:
# Convert relevant columns to numeric after removing any underscores.This was done as I saw a value in Total_Current_Loans with an underscore.
for col in ["Total_Current_Loans", "Current_Debt_Outstanding", "Income_Annual", "Credit_Limit", "Age", "Total_Credit_Cards", "Total_Bank_Accounts", "Delay_from_due_date"]:
    train_data[col] = pd.to_numeric(
        train_data[col].astype(str).str.replace("_", "", regex=False), errors="coerce"
    )
    test_data[col] = pd.to_numeric(
        test_data[col].astype(str).str.replace("_", "", regex=False), errors="coerce"
    )
train_data

,Age,Income_Annual,Base_Salary_PerMonth,Total_Bank_Accounts,Total_Credit_Cards,Rate_Of_Interest,Total_Current_Loans,Loan_Type,Delay_from_due_date,Total_Delayed_Payments,...,Credit_Mix,Current_Debt_Outstanding,Ratio_Credit_Utilization,Credit_History_Age,Payment_of_Min_Amount,Per_Month_EMI,Monthly_Investment,Payment_Behaviour,Monthly_Balance,Credit_Score
0,51,101583.480,3086.683333,5,7,10,4,"Payday Loan, Student Loan, Payday Loan, and De...",8,8,...,Standard,50.93,34.462154,24 Years and 1 Months,No,190.811017,630.0157894388726,Low_spent_Large_value_payments,314.0021934422197,Standard
1,23,101926.950,8635.912500,4,4,9,1,Mortgage Loan,13,9,...,_,1058.00,39.693812,20 Years and 5 Months,No,70.587681,662.8039273360225,Low_spent_Medium_value_payments,410.1996419555151,Standard
2,49,158871.120,3086.683333,0,4,8,1,Home Equity Loan,8,2,...,Good,576.48,39.367225,19 Years and 0 Months,No,86.905860,746.8059854204569,Low_spent_Medium_value_payments,742.5141542054829,Standard
3,40,60379.280,3086.683333,5,6,18,3,"Personal Loan, Mortgage Loan, and Credit-Build...",15,12,...,Standard,725.39,29.061701,17 Years and 1 Months,NM,90.906385,166.41865803064803,High_spent_Medium_value_payments,473.13562343490486,Standard
4,17,50050.830,4085.902500,9,10,20,5,"Student Loan, Credit-Builder Loan, Debt Consol...",28,NaN,...,Bad,3419.10,30.386321,4 Years and 6 Months,Yes,190.445060,56.789441169542684,High_spent_Large_value_payments,401.3557486786916,Poor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,55,114597.040,3086.683333,7,6,4,4,"Auto Loan, Not Specified, Payday Loan, and Stu...",32,8,...,Good,926.18,26.436313,31 Years and 9 Months,No,225.923762,327.61966834569836,High_spent_Medium_value_payments,633.131903,Poor
79996,28,8227.855,656.654583,6,8,31,-100,"Not Specified, Credit-Builder Loan, Credit-Bui...",61,20,...,_,2695.38,24.127401,11 Years and 8 Months,NM,7352.000000,49.54415830254037,Low_spent_Medium_value_payments,268.108435,Poor
79997,46,35032.660,2853.388333,10,6,33,7,"Credit-Builder Loan, Student Loan, Personal Lo...",48,18,...,Bad,1789.00,25.086176,11 Years and 5 Months,Yes,150.500097,106.73567925309915,Low_spent_Small_value_payments,318.103057,Poor
79998,42,129680.280,10643.690000,8,3,5,2,"Mortgage Loan, and Credit-Builder Loan",27,18,...,_,240.27,33.944094,20 Years and 5 Months,NM,114.165609,567.1798727916067,High_spent_Small_value_payments,643.023518,Standard


In [5]:
# Prepare training features and labels
X_train = train_data.drop(columns="Credit_Score")
y_train = train_data["Credit_Score"]

In [6]:
# Feature engineering in train data
train_data["Debt_Income_Ratio"] = (
    train_data["Current_Debt_Outstanding"] / train_data["Income_Annual"]
)
train_data["Income_Credit_Limit_Ratio"] = (
    train_data["Income_Annual"] / train_data["Credit_Limit"]
)
train_data["Debt_Credit_Limit_Ratio"] = (
    train_data["Current_Debt_Outstanding"] / train_data["Credit_Limit"]
)

In [7]:
# Replace infinity values in train data
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
train_data.fillna(train_data.median(numeric_only=True), inplace=True)

# Label encode the target variable in train data
label_encoder = LabelEncoder()
train_data["Credit_Score"] = label_encoder.fit_transform(train_data["Credit_Score"])

In [8]:
# Step 1: Identify numeric and categorical features

X_train.replace("-", np.nan, inplace=True)
X_train.replace("NM", np.nan, inplace=True)

categorical_features = [
    "Loan_Type",
    "Credit_Mix",
    "Credit_History_Age",
    "Payment_of_Min_Amount",
    "Payment_Behaviour",
]
X_train[categorical_features] = X_train[categorical_features].astype(str)

numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X_train.select_dtypes(include=["object", "category"]).columns

# Step 2: Handle Missing Values
# Use SimpleImputer to fill in missing values. Strategies: 'mean' for numerical, 'most_frequent' for categorical.

# Define the preprocessing steps for numeric features
numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="mean"))])

# Define the preprocessing steps for categorical features
# Using OneHotEncoder to convert categorical variables into binary (dummy) variables.
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "encoder",
            OneHotEncoder(handle_unknown="ignore"),
        ),  #! Can change this step for different encoding methods
    ]
)

# Step 3: Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [9]:
# Prepare test data
test_data.fillna(train_data.median(numeric_only=True), inplace=True)

# Feature engineering in test data
test_data["Debt_Income_Ratio"] = (
    test_data["Current_Debt_Outstanding"] / test_data["Income_Annual"]
)
test_data["Income_Credit_Limit_Ratio"] = (
    test_data["Income_Annual"] / test_data["Credit_Limit"]
)
test_data["Debt_Credit_Limit_Ratio"] = (
    test_data["Current_Debt_Outstanding"] / test_data["Credit_Limit"]
)

In [10]:
print(X_train.dtypes)

Age                           int64
Income_Annual               float64
Base_Salary_PerMonth        float64
Total_Bank_Accounts           int64
Total_Credit_Cards            int64
Rate_Of_Interest              int64
Total_Current_Loans           int64
Loan_Type                    object
Delay_from_due_date           int64
Total_Delayed_Payments       object
Credit_Limit                float64
Total_Credit_Enquiries      float64
Credit_Mix                   object
Current_Debt_Outstanding    float64
Ratio_Credit_Utilization    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Per_Month_EMI               float64
Monthly_Investment           object
Payment_Behaviour            object
Monthly_Balance              object
dtype: object


In [11]:
# Check for mixed data types in each column
for col in X_train.columns:
    unique_types = set(type(x) for x in X_train[col].dropna())
    if len(unique_types) > 1:
        print(f"Column '{col}' has mixed types: {unique_types}")

X_train["Monthly_Balance"] = pd.to_numeric(X_train["Monthly_Balance"], errors="coerce")
test_data["Monthly_Balance"] = pd.to_numeric(
    test_data["Monthly_Balance"], errors="coerce"
)

# Fill missing values in 'Monthly_Balance' with the median value
X_train["Monthly_Balance"].fillna(X_train["Monthly_Balance"].median(), inplace=True)
test_data["Monthly_Balance"].fillna(test_data["Monthly_Balance"].median(), inplace=True)

Column 'Monthly_Balance' has mixed types: {<class 'float'>, <class 'str'>}


C:\Users\krish\AppData\Local\Temp\ipykernel_10416\1960175337.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train["Monthly_Balance"].fillna(X_train["Monthly_Balance"].median(), inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_10416\1960175337.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [12]:
# Step 4: Apply preprocessing
# Note: For cross-validation, incorporate this step within a pipeline to prevent data leakage.
X_preprocessed = preprocessor.fit_transform(X_train)
test_data_preprocessed = preprocessor.transform(test_data)

# Step 1: Convert the sparse matrix to a DataFrame immediately after transformation
X_train_df = pd.DataFrame.sparse.from_spmatrix(X_preprocessed)
test_data_df = pd.DataFrame.sparse.from_spmatrix(test_data_preprocessed)

# Step 2: Assign column names manually
X_train_df.columns = preprocessor.get_feature_names_out()
test_data_df.columns = preprocessor.get_feature_names_out()

# Display the first few rows of the transformed DataFrame
print(X_train_df.head())
print(test_data_df.head())

# Display the first few rows of the transformed DataFrame
print(X_train_df.head())
print(test_data_df.head())

# Step 5: Train-Test Split for the preprocessed data
X_train, X_val, y_train, y_val = train_test_split(
    X_train_df, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Verify the transformations
print("Transformed Training Data Shape:", X_train.shape)
print("Transformed Validation Data Shape:", X_val.shape)
print("Transformed Test Data Shape:", test_data_preprocessed.shape)

   num__Age  num__Income_Annual  num__Base_Salary_PerMonth  \
0      51.0           101583.48                3086.683333   
1      23.0           101926.95                  8635.9125   
2      49.0           158871.12                3086.683333   
3      40.0            60379.28                3086.683333   
4      17.0            50050.83                  4085.9025   

   num__Total_Bank_Accounts  num__Total_Credit_Cards  num__Rate_Of_Interest  \
0                       5.0                      7.0                   10.0   
1                       4.0                      4.0                    9.0   
2                         0                      4.0                    8.0   
3                       5.0                      6.0                   18.0   
4                       9.0                     10.0                   20.0   

   num__Total_Current_Loans  num__Delay_from_due_date  num__Credit_Limit  \
0                       4.0                       8.0               2.89   


In [16]:
models = {}
model = RandomForestClassifier(n_estimators=300, max_depth=6, random_state=42)
models["Random forest"] = model
model = XGBClassifier(
    learning_rate=0.05,
    max_depth=6,
    n_estimators=300,
    random_state=42,
    eval_metric="mlogloss",
)
models["XGBoost"] = model
model = KNeighborsClassifier(n_neighbors=5)
models["K nearest neighbours"] = model
model = LogisticRegression(random_state=42, max_iter=500)
models["Logistic regression"] = model
model = DecisionTreeClassifier(max_depth=6, random_state=42)
models["Decision tree classifier"] = model
model = GaussianNB()
models["Gaussian"] = model
base_estimator = DecisionTreeClassifier(max_depth=6)
model = AdaBoostClassifier(estimator=base_estimator, n_estimators=300, random_state=42)
models["Adaboost"] = model

In [17]:
print(y_train.head())

30544        Poor
41745        Poor
28857    Standard
34158        Poor
39129        Good
Name: Credit_Score, dtype: object


In [18]:
for key, value in models.items():
    try:
        # Define model pipeline
        pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", value)])

        # Fit the pipeline on training data
        pipeline.fit(X_train, y_train)

        test_data_transformed = pipeline.transform(test_data_df)
        # Make predictions on test data
        test_predictions = pipeline.predict(test_data_transformed)

        # Convert predictions back to original labels
        test_predictions_labels = label_encoder.inverse_transform(test_predictions)
        # test_predictions_encoded = label_encoder.transform(test_predictions)
        # Prepare the submission file
        submission = pd.DataFrame(
            {"ID": test_ids, "Credit_Score": test_predictions_labels}
        )
        submission.to_csv(f"submission_{key}.csv", index=False)

        print(f"Submission file 'submission_{key}.csv' created successfully!")
    except Exception as e:
        print(f"Error : {e}")

Error : A given column is not a column of the dataframe
Error : A given column is not a column of the dataframe
Error : A given column is not a column of the dataframe
Error : A given column is not a column of the dataframe
Error : A given column is not a column of the dataframe
Error : A given column is not a column of the dataframe
Error : A given column is not a column of the dataframe


In [19]:
# Define models
models = {
    "Random forest": RandomForestClassifier(
        n_estimators=300, max_depth=6, random_state=42
    ),
    "XGBoost": XGBClassifier(
        learning_rate=0.05,
        max_depth=6,
        n_estimators=300,
        random_state=42,
        eval_metric="mlogloss",
    ),
    "K nearest neighbours": KNeighborsClassifier(n_neighbors=5),
    "Logistic regression": LogisticRegression(random_state=42, max_iter=500),
    "Decision tree classifier": DecisionTreeClassifier(max_depth=6, random_state=42),
    "Gaussian": GaussianNB(),
    "Adaboost": AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=6), n_estimators=300, random_state=42
    ),
}

# Loop over models to create submission files
for key, model in models.items():
    try:
        # Define model pipeline
        pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", model)])

        # Fit the pipeline on training data
        pipeline.fit(X_train, y_train)

        # Make predictions on test data
        test_predictions = pipeline.predict(test_data)

        # Convert predictions back to original labels
        test_predictions_labels = label_encoder.inverse_transform(test_predictions)

        # Prepare the submission file
        submission = pd.DataFrame(
            {"ID": test_ids, "Credit_Score": test_predictions_labels}
        )
        submission.to_csv(f"submission_{key}.csv", index=False)

        print(f"Submission file 'submission_{key}.csv' created successfully!")
    except Exception as e:
        print(f"Error for model '{key}': {e}")

Error for model 'Random forest': A given column is not a column of the dataframe
Error for model 'XGBoost': A given column is not a column of the dataframe
Error for model 'K nearest neighbours': A given column is not a column of the dataframe
Error for model 'Logistic regression': A given column is not a column of the dataframe
Error for model 'Decision tree classifier': A given column is not a column of the dataframe
Error for model 'Gaussian': A given column is not a column of the dataframe
Error for model 'Adaboost': A given column is not a column of the dataframe
